# Exploring ECCC prediction products with the MSC's Geospatial Web Services

During this tutorial we'll explore the mechanics of retrieving prediction data from ECCC's Meteorological Service of Canada (MSC). We'll take a close look at three geospatial web services, the Web Map Service (WMS), the Web Coverage Service (WCS), and the Open Geospatial Consortium's (OGC) Features API. All three of these web services are application programming interfaces that allow us to "talk with" and receive data from the MSC's GeoMet service. In this tutorial we'll focus on retrieving precipitation analysis predictions and observed precipitation accumulations, but the "recipes" below apply to any datasets available from MSC's GeoMet. 

It's important to note that the code that we'll explore here is analagous to the code that your FEWS system uses to retreive data from the MSC. As such, the goals for this tutorial are to give you a sense of what is going on "under the hood" of your FEWS system and to familiarize you with the terminology and process of retrieving data from the MSC's geospatial web services. By the end of this tutorial you will understand how scripts can be used to retrieve images / data from the MSC and how these scripts can be used to manipulate raw prediction data to create maps and plots. Given that your FEWS system does the work of the code shown in this tutorial, it's not necessary to understand the minutae of the Python code blocks below. Rather, this tutorial is intended to provide you with general background information that you may find helpful when having technical discussions with your industry partners. The information in this tutorial may also prove useful for exploring MSC data further, and for the technically savvy, this tutorial can be a jumping off point for writing or contributing to related code.

Finally, it's worth noting that your FEWS system likely fetches the majority of data from GeoMet's sister service, Datamart. Unfortunately, we won't explore Datamart during this tutorial for technical reasons (the data retrieval mechanism is hard to use in a "lightweight" tutorial like this, especially with conference centre WiFi!). However, the mechanics and process of querying for data from the two services is similar and the differences are not important for the purposes of this tutorial.


## Setting things up

In [ ]:
# @title Downloading tutorial requirements into this Colab instance

# The following `%%capture` command will hide a bunch of text that gets
# generated when we install the following libraries
%%capture
!pip install owslib
!pip install cartopy
!pip install panel hvplot jupyter_bokeh

In [ ]:
# @title Setting up plotting environment
import holoviews as hv
import hvplot.pandas
import panel as pn
pn.extension()

if 'google.colab' in str(get_ipython()):
    def _render(self, **kw):
        hv.extension('bokeh')
        return hv.Store.render(self)
    hv.core.Dimensioned._repr_mimebundle_ = _render

In [ ]:
# @title Importing tutorial requirements

# Tools for handling data
import re
import warnings
import xarray as xr
import pandas as pd
import geopandas as gpd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
from datetime import datetime, timedelta

# Tools that will help us connect to GeoMet's geospatial web services
from owslib.wms import WebMapService
from owslib.wcs import WebCoverageService
from owslib.wcs import Authentication
from owslib.ogcapi.features import Features
from owslib.ogcapi.coverages import Coverages

# Plotting tools
import cartopy.crs as ccrs
import cartopy.feature as cf
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib import cm
from matplotlib.animation import FuncAnimation
from matplotlib.gridspec import GridSpec
from IPython.display import Image, display, clear_output
import hvplot.xarray

from matplotlib import rc
rc('animation', html='jshtml')

# Here we'll set up a filter so we don't see a bunch of warnings
warnings.filterwarnings('ignore')

In this tutorial we'll be viewing prediction data from the MSC's geospatial web services. This will involve making queries for data from a specific location in Canada. We'll define our location using a bounding box and we'll center our bounding box on a location of interest. You're welcome to center your bounding box on a location of your choosing, like your favourite gauge station, your office, your go-to camping spot, etc.

To generate your bounding box, replace the values for the `lat` and `lon` variables below. To get the lat / lon coordinates of your place of interest, you can use this website: https://www.latlong.net/convert-address-to-lat-long.html

In [ ]:
# @title Setting our location of interest

# Lat / Lon coords for place of interest
lat = 
lon = 

# We'll make our bounding box 5 x 5 degrees
buffer = 5

# Bounding box for the Web Map Service
bbox_wms = (lon-buffer, lat-buffer*0.5, lon+buffer, lat+buffer*0.5)

# Subset (bounding box) for the Web Coverage Service
bbox_wcs = [('lat', lat-buffer, lat+buffer), ('lon', lon-buffer, lon+buffer)]

## Viewing current precipitation using GeoMet's Web Map Service

First, we'll start by creating an object, `wms` that will allow us to connect to GeoMet's web map service (WMS). 

In [ ]:
# @title Creating a connection to the WMS

# Create a connector object to query the Web Map Service
wms = WebMapService(
    f'https://geo.weather.gc.ca/geomet?&SERVICE=WMS',
    version='1.3.0',
    timeout=300
)

Next, we'll query for a map image from the High Resolution Determinisitic Precipitation Analysis (HRDPA). But before we can do that, we'll need to gather the information we need to construct our query. We can get this information from two places, 1) the WMS documentation and 2) our `wms` connection object. Starting with the WMS documentation we know we'll need the following info:

- the name of the HRDPA layer
- the style of the layer, or in other words, the colorbar for the map image
- the format we'd like to receive the data in
- the coordinate reference system that the data should be projected onto
- the bounding box that the WMS will use to cut out the subset of the grid we're interested in

From the documentation, we can see that to get a map image, we'll need to specify the data format as `image/jpeg` and we can use the default coordinate reference system, `EPSG:4326`. We defined our bounding box above, so that leaves the layer name and layer style for us to find. To get this information, we'll need to poke around with the `wms` connection object, or we can view the layer on AniMet.

Let's explore the metadata with the `wms` connection object. First, we'll print all the available layers that contain the word `HRDPA`.

In [ ]:
# Print HRDPA layer names from the WMS connector
for layer_name in wms.contents.keys():
  if  in layer_name:
    print(layer_name)

Some of these layer names look a bit cryptic. We can choose to print a layer's title, which should give us a better indication of what the layer contains. For the first part of this tutorial, we'll explore the `HRDPA.6F_PR` layer.

In [ ]:
# The layer we're going to query in this section
layer = 

In [ ]:
# Check the layer title for additional detail
wms[layer].title

Now let's look for the available colourbars (layer styles) we can use for the map image.

In [ ]:
# Check out the colourmap options for our layer
for style, details in wms[layer].styles.items():
  print(
      f"Style name: {style} \n Link: {details['legend']}"
  )

In [ ]:
# The default colourmap that we're going to specify for our layer
hrdpa_style = 

Now we have the minimum information necessary to make a query. Let's make a query for a map image using the `wms` object and then we'll use Python's `Image` function to parse the information from the query and display the map. 

To get our requested information, the `wms` object takes our query parameters and formats them into a URL. The `wms` object URL tells GeoMet what information it should prepare for us. Then, the `wms` object requests the information that GeoMet prepared.

In [ ]:
# @title Making a map query

width = 500
height = 250

# Querying the WMS for a subset of our layer defined by our bounding box
response = wms.getmap(
    layers=[],
    format='image/jpeg',
    srs='EPSG:4326',
    bbox=,
    size=(width, height), # this defines the size of the map image
    styles=[],
)

# Show image
Image(response.read(), width=width, height=height)

Et voila! Precipitation accumulations should show up as colours on a black background. If you only see a black image, this is normal - it just means there isn't any accumulated precipitation in your bounding box for the timestep returned from the query (more on time in a moment...) 

Now that we've made our first query, let's have a look at the query URL.

In [ ]:
# The raw HTML for our WMS query
response.geturl()

Just like the `wms` object, your FEWS system would use an analogous process to construct URLS and query GeoMet. The code that we're playing around with here is similar to the code "under the hood" of your FEWS system.

Let's explore some other aspects of making queries to GeoMet. Previously, we made a query for a map image with just the minimum amount of necessary information. Now you might be wondering what time the map image was issued. We can go back to our `wms` object to learn more about the time metadata for the HRDPA layer that we queried.

In [ ]:
# Pull out the time metadata
time_info = wms[layer].dimensions['time']
time_info

From the information about, we can first see a `default` time. When we don't specify any time information in our GeoMet queries, it will serve us the layer associated with the `default` time. The `default` time is typically the most recent forecast or analysis issue. Next, we can see several timestamps associated with the time `values`. This is a bit hard to read, so let's use Python to make this information more legible.

In [ ]:
# @title Exploring forecast issue and timestamp metadata

start, end, issue_interval = time_info['values'][0].split('/')
print(f"First query-able time: {start}")
print(f"End of query window: {end}")
print(f"Interval that queries can be made: {issue_interval[-3:]}")

The time `values` reveal the window of time of currently available analyses for the HRDPA layer. In other words, we can query for analysis issues starting at the `start` time and make queries at six hour intervals until the `end` time. Note that the `default` time is the same as the `end` time. 

Now, let's make another map image query, but this time we'll specify an analysis issue.

In [ ]:
width = 500
height = 250

# Querying the WMS for a subset of our layer defined by our bounding box
response = wms.getmap(
    layers=[layer],
    format='image/jpeg',
    srs='EPSG:4326',
    bbox=bbox_wms,
    size=(width, height),
    styles=[hrdpa_style],
    time=
)

# Show image
Image(response.read(), width=width, height=height)

And here is our new image! 

All this said, these images are a bit boring - we don't have any map features, or a colour bar. Making WMS queries like this with Python is a bit limited. But, if we query GeoMet for the raw data, instead of just a map image, we'll have much more freedom to create a custom map. Let's try this using GeoMet's Web Coverage Service (WCS).

## Manipulating current precipitation using GeoMet's Web Coverage Service

Just like we did above for the WMS, we'll start by creating an object, `wcs` that will allow us to connect to the WCS. 

In [ ]:
# @title Creating a connection to the WCS

# Create a connector object to query the Web Map Service
wcs = WebCoverageService(
    f'https://geo.weather.gc.ca/geomet?&SERVICE=WCS&COVERAGEID',
    version='2.0.1',
    timeout=300
)

Fortunately, making WCS queries is very similar to making WMS queries. Let's try a query using the same parameters we specifed for our WMS queries. This time, we won't use the `Image` function to generate a map. Instead, we'll store the requested information in a dataset object.

In [ ]:
# @title Making a coverage query

response = wcs.getCoverage(
    identifier=[layer],
    format='image/netcdf',
    subsettingcrs='EPSG:4326', # note the small differences in parameter names between the WMS and WCS
    subsets=bbox_wcs, # note the small differences in parameter names between the WMS and WCS 
)

# Read data into a dataset object
ds = xr.open_dataset(response.read()).load()
ds

Now let's generate a custom plot. Note that by default the plotting code is hidden. For the purpose of this tutorial the details of the plotting code aren't important other than to know that we have much more flexibility to manipulate information from GeoMet if we grab the raw data. If you're curious you can use the arrow to the left of the code block to reveal the plotting code. 

And again, just like the code here, your FEWS system fetches raw data from the MSC servers and uses code to create custom visuals.

In [ ]:
#@title Plot: 6h precipitation amounts from HRDPA

# Create custom colour-map for the HDRPA precip data
min_precip = 0
max_precip = 90
interval = 30

# more granularity for smaller precip accumulations
lower_levels = [0.1, 0.5, 1, 2.5, 5, 10, 15, 20, 25]
# less granularity for the larger precip accumulations
upper_levels = list(range(30, max_precip+interval, interval))

levels = lower_levels + upper_levels

cmap = plt.get_cmap("turbo").copy() # copy an existing color-map
cmap.set_over("maroon") # set the colour for the maximum level
cmap.set_under("beige") # set the colour for the minimum level

# Create figure box
fig = plt.figure(figsize=(6,8))

# Specify coordinate reference system that the data will be projected to
projection = ccrs.LambertConformal()
ax = plt.axes(projection=projection)

west, south, east, north = bbox_wms
ax.set_extent([west, east, south, north])

# Add HRDPA data to map

x = ds['Band1']['lon']
y = ds['Band1']['lat']

mesh = ax.pcolormesh(
    x, y, ds['Band1'],
    transform=ccrs.PlateCarree(),
    cmap=cmap,
    norm=mcolors.BoundaryNorm(levels, ncolors=256, extend='both'),
    label='none',
    shading='auto'
)

# Add geographic features to map

edgecolor = 'dimgrey'
linewidth = 0.5

# Physical features
ax.add_feature(cf.COASTLINE, edgecolor=edgecolor, linewidth=linewidth)
ax.add_feature(cf.LAND, edgecolor=edgecolor, facecolor="beige", linewidth=linewidth)
rivers = cf.NaturalEarthFeature(
    category='physical',
    name='rivers_lake_centerlines',
    scale='10m',
    linewidth=0.5,
    edgecolor=edgecolor,
    facecolor='none',
)
ax.add_feature(rivers)
ax.add_feature(cf.LAKES, facecolor='white', edgecolor=edgecolor, linewidth=linewidth)

# Country borders
countries = cf.NaturalEarthFeature(
    category='cultural',
    name='admin_0_countries_lakes',
    scale='10m',
    edgecolor=edgecolor,
    facecolor='none',
    linewidth=linewidth
)
ax.add_feature(countries)

# Province and Territory borders
borders = cf.NaturalEarthFeature(
    category='cultural',
    name='admin_1_states_provinces_lines',
    scale='10m',
    edgecolor=edgecolor,
    facecolor='none'
)
ax.add_feature(borders, linestyle='dashed')

# Add colorbar as separate axis and make it the right size
cb = fig.colorbar(
    mesh,
    location='bottom',
    shrink=1.0,
    pad=0.025,
    fraction=0.05,
    aspect=30,
    label='6-hr total precipitation (mm)'
)
# cax = fig.add_axes([ax.get_position().x1+0.01,ax.get_position().y0,0.02,ax.get_position().height])
# cb.cax.set_yticklabels(["{:.0f}".format(i) if i >= 1 else "{:.1f}".format(i) for i in cb.get_ticks()])

# Add a plot title
ax.set_title(
      f"Latest HRDPA 6-hr precip amount [mm]"
  );

Now our map image looks more interesting!

Just like with our first WMS query, we didn't specify any time information so our map was created using the latest analysis issue. Let's take a look to see what a WCS query looks like when we specify the time information. First, we'll load our time metadata from the WMS again.

**Note: the metadata and time information for a given GeoMet layer is only available from the WMS. If you want to only make WCS queries, you will still need to create a `wms` object to look for the necessary metadata and time information.

In [ ]:
# Recall that metadata (including time info) comes from the WMS:
# time_info = wms[layer].dimensions['time']

start, end, issue_interval = time_info['values'][0].split('/')
print(f"First query-able time: {start}")
print(f"End of query window: {end}")
print(f"Interval that queries can be made: {issue_interval[-3:]}")

Rather than querying for a specific issue time and making the same static map image again, let's query for multiple issue times and create a map animation. First, we'll use the time `values` to construct a list of all available analysis issue times. 

In [ ]:
first_time = start
last_time = end

iso_format = "%Y-%m-%dT%H:%M:%SZ"

# Convert date strings to datetime objects
first = datetime.strptime(first_time, iso_format)
last = datetime.strptime(last_time, iso_format)

# Remove anything that isn't a number from the datetime interval (time between issues)
intvl = int(re.sub(r'\D', '', issue_interval))

# Create a list of forecast datetimes
hrs = [first]
while first < last:
  first = first + timedelta(hours=intvl)
  hrs.append(first)

# create a list of ISO formatted datetime strings
analysis_hrs = [datetime.strftime(hr, iso_format) for hr in hrs if hr < last]

Now, we'll step through our list of available issue times and make a query for each time. Note that we need to make separate queries to GeoMet for each issue time. In other words, we can't make a query to GeoMet to return multiple layers at once.

Depending on our WiFi speed, this next code block may take a minute or two to run. Let's run the code block and pause for questions while we wait for our queries to finish.

In [ ]:
# @title Querying for multiple predictions
arrys = []

for i in range(len(analysis_hrs)):

    clear_output(wait=True)
    display(f"Querying for layer at time {analysis_hrs[i]}")

    response = wcs.getCoverage(
        identifier=[layer],
        format='image/netcdf',
        subsettingcrs='EPSG:4326',
        subsets=bbox_wcs,
        TIME=
    )

    # read into an xarray
    ds = xr.open_dataset(response.read()).load()

    # add the time metadata as a new dimension and coordinate
    da = ds.expand_dims(time=[datetime.strptime(analysis_hrs[i], iso_format)])

    # append to list of xarrays
    arrys.append(da)

fcasts = xr.concat([ds for ds in arrys], dim='time')

Finally, we're ready to create our animation. Just like above, the details of this code are hidden for simplicity. This code might also take a few minutes to finish and in the interest of time, we'll only animate a few layers (issue times).

In [ ]:
#@title Animation: 6h precipitation amounts from HRDPA
# Create figure box
gs = GridSpec(1, 2, width_ratios = [0.9, 0.05])
fig = plt.figure(figsize = (7, 7))

# fig.subplots_adjust(left=0, bottom=0, right=1, top=1)

# Specify coordinate reference system that the data will be projected to
projection = ccrs.LambertConformal()
ax = fig.add_subplot(gs[0], projection=projection)
cbar_ax = fig.add_subplot(gs[1])

# shrink the colorbar so that it has the same height as the plot
b = cbar_ax.get_position()
new_h = b.height*0.75
pad = (b.height-new_h)/2.
new_y0 = b.y0 + pad
new_y1 = b.y1 - pad
b.y0 = new_y0
b.y1 = new_y1
cbar_ax.set_position(b)

west, south, east, north = bbox_wms
ax.set_extent([west, east, south, north])

x = fcasts['Band1']['lon']
y = fcasts['Band1']['lat']

# Add geographic features to map

edgecolor = 'dimgrey'

# Physical features
ax.add_feature(cf.COASTLINE, edgecolor=edgecolor)
ax.add_feature(cf.LAND, edgecolor=edgecolor, facecolor="beige")
rivers = cf.NaturalEarthFeature(
    category='physical',
    name='rivers_lake_centerlines',
    scale='10m',
    linewidth=0.5,
    edgecolor=edgecolor,
    facecolor='none',
)
ax.add_feature(rivers)
ax.add_feature(cf.LAKES, facecolor='white', edgecolor=edgecolor)

# Country borders
countries = cf.NaturalEarthFeature(
    category='cultural',
    name='admin_0_countries_lakes',
    scale='10m',
    edgecolor=edgecolor,
    facecolor='none'
)
ax.add_feature(countries)

# Province and Territory borders
borders = cf.NaturalEarthFeature(
    category='cultural',
    name='admin_1_states_provinces_lines',
    scale='10m',
    edgecolor=edgecolor,
    facecolor='none'
)
ax.add_feature(borders, linestyle='dashed')

mesh = ax.pcolormesh(
      x, y, fcasts['Band1'].sel(time=hrs[0]),
      transform=ccrs.PlateCarree(),
      cmap=cmap, norm=mcolors.BoundaryNorm(levels, ncolors=256, extend='both'),
      label='none', shading='auto'
  )
cbar_ax.set_yticklabels(["{:.0f}".format(i) if i >= 1 else "{:.1f}".format(i) for i in cb.get_ticks()])
fig.colorbar(mesh, cax=cbar_ax, ticks=levels, label='6-hr total precipitation (mm)')


def update_map(i):
  mesh = ax.pcolormesh(
      x, y, fcasts['Band1'].sel(time=hrs[i]),
      transform=ccrs.PlateCarree(),
      cmap=cmap, norm=mcolors.BoundaryNorm(levels, ncolors=256, extend='both'),
      label='none', shading='auto'
  )
  ax.set_title(
      f"HRDPA 6-hr precip amount [mm] valid for {analysis_hrs[i]} UTC"
  )

anim = FuncAnimation(fig, update_map, frames=8, interval=600, repeat=False)
plt.close()
anim

And there you have it! We've finished running through the basics of the WMS and WCS APIs. We've seen how we can use the WMS to retrieve metadata and generate simple map images to give us a "first look" at GeoMet layers. And we've seen how we can manipulate and customize info from GeoMet by retrieving the raw data from the WCS. We've also learned that the code here is analogous to the functionality of your FEWS system that retrieves data from the MSC. 

To round out this tutorial, we'll have a quick look at GeoMet's newer APIs.

## Viewing forecasted precipitation using GeoMet's OGC-Features API

The Web Map and Web Coverage services are GeoMet's mature geospatial web services, but more recently, GeoMet has offered data from a suite of newer services, the OGC APIs. In this tutorial, we'll explore the OGC-Features API. The goal for this next section will be to compare the HRDPA precipitation amounts we retrieved from the WCS earlier with observed precipitation amounts from a climate station close to the location of interest that we specified at the beginning of this tutorial. We'll use the OGC-Features API to grab the precipiation observations.

If your original location didn't have much going on in terms of precipitation, you can change the coordinates in the box below.

In [ ]:
# Lat / Lon coords for place of interest
lat = 45.48
lon = -73.79

buffer = 1.0

# Bounding box for the Web Map Service
bbox_oafeat = (lon-buffer, lat-buffer*0.5, lon+buffer, lat+buffer*0.5)

To start let's create a OGC-Features API connection, which we'll call `oafeat`. Next, we'll make two queries. First, we'll query the collection of climate stations to find a station that has real-time data. Then, we'll use the climate station ID to request data from the station. Unlike, the WMS and WCS, both the metadata and data are available via the OGC-Features API. Another bonus of the OGC-Features API is that we can preview the available data and explore its metadata in our browser. This makes it easier to find the query information.

In [ ]:
# @title Creating a connection to the OGC-Features API and making a query

oafeat = Features("https://api.weather.gc.ca/")

# Make a query for climate station metadata
stations = oafeat.collection_items(
    'climate-stations', # here we're specifying that we'll query the collection of climate stations
    bbox=bbox_oafeat,
)

Let's store the information from the Features API in a table.

In [ ]:
station_table = gpd.GeoDataFrame.from_features(stations['features'])
station_table.head(10)

Using the information from the table above, pick a station with real-time data and enter its station ID into the code block below as the variable `stn_id`.

In [ ]:
stn_id = 

# We'll grab data as far back as the first available analysis time until the latest available time
start = start 
end = station_table['HLY_LAST_DATE'].loc[station_table['STN_ID'] == stn_id]

# Filter the table so that we remove all unneeded stations
stn = station_table.loc[station_table['STN_ID'] == stn_id]

# Pull out the station latitude and longitude
stn_lon = stn['geometry'].x.values[0]
stn_lat = stn['geometry'].y.values[0]

In [ ]:
# Make a query for data from a climate station
weather = oafeat.collection_items(
    
    bbox=
    STN_ID= # we'll filter our query by our station of interest
    datetime=f"{start}/{end}" # and we'll filter by our specified time period
)
# again, we'll store this information in a table
weather_table = gpd.GeoDataFrame.from_features(weather['features'])
weather_table.head()

In [ ]:
# Sanity check that this station is recording precipitation
weather_table['PRECIP_AMOUNT'].sum()

Now that we've fetched our precipitation observations, we'll clean up our data table to make manipulating and plotting the data easier.

In [ ]:
# Remove all columns aside from the time and precip amount
precip = weather_table[['UTC_DATE', 'PRECIP_AMOUNT']]

# Rename the columns for nice plot titles
precip = precip.rename(columns={'UTC_DATE': 'Time', 'PRECIP_AMOUNT': 'Observations'})

# Index our table by time so that we can put our data in the proper order
precip = precip.set_index('Time')
precip.index = pd.to_datetime(precip.index)
precip.sort_index(inplace=True)

Since we want to compare the precipitation observations with the data from HRDPA, we will need to compute the observed 6-hr accumulations.

In [ ]:
# Aggregate the hourly precipitation amounts over 6-hour intervals
precip = precip.resample(rule='6H', closed='right', label='right').sum()

In [ ]:
# Print the stop of the precipitation table to take a peak at the data
precip.head()

Let's query for the HRDPA data again from the WCS just in case the available times changed or if we changed our location of interest. Let's also query for data from the Regional Deterministic Precipitation Analysis (RDPA) to add to our comparison. Since we've seen this code above, we've hidden it here for simplicity.

In [ ]:
# @title Querying for model data for comparison
buffer = 5
bbox_wcs = [('lat', lat-buffer, lat+buffer), ('lon', lon-buffer, lon+buffer)]

arrys = []

for i in range(len(analysis_hrs)):

    clear_output(wait=True)
    display(f"Querying for HRDPA layer at time {analysis_hrs[i]}")

    response = wcs.getCoverage(
        identifier=['HRDPA.6F_PR'],
        format='image/netcdf',
        subsettingcrs='EPSG:4326',
        subsets=bbox_wcs,
        TIME=analysis_hrs[i]
    )

    # read into an xarray
    ds = xr.open_dataset(response.read()).load()

    # add the time metadata as a new dimension and coordinate
    da = ds.expand_dims(time=[datetime.strptime(analysis_hrs[i], iso_format)])

    # append to list of xarrays
    arrys.append(da)

hrdpa_fcasts = xr.concat([ds for ds in arrys], dim='time')

arrys = []

for i in range(len(analysis_hrs)):

    clear_output(wait=True)
    display(f"Querying for RDPA layer at time {analysis_hrs[i]}")

    response = wcs.getCoverage(
        identifier=['RDPA.6F_PR'],
        format='image/netcdf',
        subsettingcrs='EPSG:4326',
        subsets=bbox_wcs,
        TIME=analysis_hrs[i]
    )

    # read into an xarray
    ds = xr.open_dataset(response.read()).load()

    # add the time metadata as a new dimension and coordinate
    da = ds.expand_dims(time=[datetime.strptime(analysis_hrs[i], iso_format)])

    # append to list of xarrays
    arrys.append(da)

rdpa_fcasts = xr.concat([ds for ds in arrys], dim='time')

Just like we cleaned up the observations, we'll also clean up the analysis data. Importantly, when we query a layer from GeoMet, it is returned to us with a generic name, `Band1`. We'll want to change the `Band1` name so we can differentiate the HRDPA and RDPA data.

In [ ]:
# Select only the grid cell that contains or is closest to our climate station
hrdpa = hrdpa_fcasts.sel(lat=stn_lat, lon=stn_lon, method='nearest')
# Move the HRDPA data into a dataframe
hrdpa = hrdpa.to_dataframe().drop(columns=['lat', 'lon', 'crs'])
# Rename the `Band1` column to something more prescriptive
hrdpa = hrdpa.rename(columns={'time': 'Time', 'Band1': 'HRDPA'})
hrdpa.head()

In [ ]:
# We'll do the same cleanup with the RDPA data
rdpa = rdpa_fcasts.sel(lat=stn_lat, lon=stn_lon, method='nearest')
rdpa = rdpa.to_dataframe().drop(columns=['lat', 'lon', 'crs'])
rdpa = rdpa.rename(columns={'time': 'Time', 'Band1': 'RDPA'})
rdpa.head()

Now that our data cleanup is finished, we can merge the observed and analysis data into one table.

In [ ]:
# Combine the HRDPA, RDPA, and observed precip data into one table
precip = precip.join(hrdpa)
precip = precip.join(rdpa)
precip.head()

Let's also take the cumulative sum of the precipitation amounts over the whole period so that we can visualize the data in a couple of different ways.

In [ ]:
# Compute the cumulative precipitation amounts for each precip time series
precip['Cumulative Observations'] = precip['Observations'].cumsum()
precip['Cumulative HRDPA'] = precip['HRDPA'].cumsum()
precip['Cumulative RDPA'] = precip['RDPA'].cumsum()

Finally, let's plot a time series of 6-hour precipitation amounts and let's create a small table to the right of the plot so we can explore the data values. How well does the prediction and observed data compare? 

In [ ]:
(precip.hvplot.step(x='Time', y=['Observations', 'HRDPA', 'RDPA'], value_label='6-hr Precipitation amounts (mm)') +
 precip.hvplot.table(['Time', 'Observations', 'HRDPA', 'RDPA']))

We can compare the observed data and prediction data in another way by looking at the cumulative amounts over the entire period.

In [ ]:
(precip.hvplot.step(x='Time', y=['Cumulative Observations', 'Cumulative HRDPA', 'Cumulative RDPA'], value_label='Cumulative precipitation (mm)') +
 precip.hvplot.table(['Time', 'Cumulative Observations', 'Cumulative HRDPA', 'Cumulative RDPA']))

And that's a wrap! In this section we learned how queries are made to the newer GeoMet API, OGC-Features. We saw that the mechanics of the queries were similar to the way we query the WMS and WCS, but that the Features API provides more convenient ways to explore the data and associated metadata. Note that hydrometric observations are available via GeoMet using the OGC-Features API.

## Summary

Overall, this tutorial provides a brief technical overview of retrieving prediction data from the MSC's GeoMet. While the mechanics of querying for data from GeoMet appear straightforward here, we still need a fair amount of requisite knowledge and coding ability to make requests and manipulate the prediction information. A data management system like FEWS removes or lessens these technical requirements. Notwithstanding, tutorials like these can help FEWS users and others become generally familiar with the way data is retrieved from GeoMet and they can provide helpful background information for technical discussions with industry partners. Also, for FEWS users who would like a hand in managing or adding to their FEWS configuration, tutorials like these can provide a gentle introduction to the various GeoMet APIs and point users to associated documentation.